In [3]:
#import all of the packages
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
import string

In [12]:
#WEBCRAWLER FUNCTION

def webCrawler():

    #define the starting location and create the deque:
    starting_link = "https://people.ischool.berkeley.edu/~chuang/i206/b5/index.html"
    html_shortcode = re.compile(r'https://people.ischool.berkeley.edu/~chuang/i206/b5/(.*)')
    queue = deque(html_shortcode.findall(starting_link))

    #save the rest of the link to concatenate later
    link_base = "https://people.ischool.berkeley.edu/~chuang/i206/b5/"
    links = re.compile(r'<a href="(.*?)">')

    #count the pages being crawled (iterate through the deque)
    count = 0
    all_links = 0 #don't have to include index.html as the first page as a link. 

    #the meat
    while True:
        #open each web page in the deque and read it. 
        webUrl = urllib.request.urlopen(link_base+queue[count])
        data = webUrl.read()

        #find the links
        found_links = links.findall(str(data))

        #add the count of the links found to the running links
        all_links += len(found_links)

        #for each of the links found on this page
        for i in range(0,len(found_links)):
            
            #if the link is not in the deque, add it 
            if found_links[i] not in queue:
                queue.append(found_links[i])

        #iterate the counter
        count+=1

        #if the count is now longer than the queue, then break out
        if count+1 > len(queue):
            break
    
    #return the list of pages, the number of pages crawled, the number of unique links found and the total number of links (non-unique)
    return "List of Pages: ", queue, "Pages Crawled: ", (count), "Total Number of Unique Links Found: ", len(queue), "Total Links (non-unique): ", all_links

    

    


In [13]:
#print the answer
answer = webCrawler()
print(answer)

#save the deque data separately from the string
deque_data = answer[1]
print(deque_data)

('List of Pages: ', deque(['index.html', 'information.html', 'Berkeley.html', 'ISchool.html', 'MIMS.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html', 'SouthHall.html', 'Campanile.html']), 'Pages Crawled: ', 10, 'Total Number of Unique Links Found: ', 10, 'Total Links (non-unique): ', 22)
deque(['index.html', 'information.html', 'Berkeley.html', 'ISchool.html', 'MIMS.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html', 'SouthHall.html', 'Campanile.html'])


In [14]:
#webCrawlIndexer Function

def webCrawlIndexer(deque_data):
    #create the dictionary
    index = {}

    #save the link base
    link_base = "https://people.ischool.berkeley.edu/~chuang/i206/b5/"

    #pull the text from the document and clean it.
    for i in range(0,len(deque_data)):
        link = str(deque_data[i])
        webUrl = urllib.request.urlopen(str(link_base)+link)
        doc = BeautifulSoup(webUrl.read(),"html.parser")
        text = doc.get_text().split()
        for j in range(0,len(text)):
            word = text[j]
            word = word.translate(str.maketrans('\n',' ', string.punctuation)).lower()

    #with the cleaned words listed if it's in the dictionary
            if word in index:
                
                #save the values for that word key
                values_list = index[word]

                #if the link is already there, skip it
                if link in values_list:
                    continue

                #if the link is not there, add it to the values list and update the dictionary pairing
                else:
                    values_list.append(link)
                    index.update({word:values_list})

    #if the cleaned word is not in the dictionary
            else:

                #create a list to create the key value pair and add the link
                values_list = []
                values_list.append(link)
                index[word] = values_list
                
    #print length of the index and save the dictionary to whatever variable the function is set to:
    print("The index contains "+str(len(index))+ " words.")
    return index

In [16]:
#execute the function
dict = webCrawlIndexer(deque_data)

#show the dictionary
print(dict)

The index contains 461 words.
{'206': ['index.html'], 'crawler': ['index.html'], 'home': ['index.html', 'information.html', 'Berkeley.html', 'ISchool.html', 'MIMS.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html', 'SouthHall.html', 'Campanile.html'], 'page': ['index.html', 'information.html'], 'the': ['index.html', 'information.html', 'ISchool.html', 'MIMS.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html', 'SouthHall.html', 'Campanile.html'], 'where': ['index.html'], 'any': ['index.html', 'UCBerkeley.html'], 'information': ['index.html', 'information.html', 'ISchool.html', 'MIMS.html'], 'can': ['index.html', 'Berkeley.html'], 'be': ['index.html', 'Berkeley.html'], 'found': ['index.html'], 'is': ['index.html', 'ISchool.html', 'MIMS.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html', 'SouthHall.html', 'Campanile.html'], 'homepage': ['index.html'], 'your': ['index.html'], 'will': ['index.html'], 'start': ['index.html'], 'fol

In [26]:
#Search Query Interface

def SearchQueryInterface():
    #set the value 
    query = None
    
    while query != "q":
        #take the user input
        query = str(input("What word would you like to search for? \n Enter q to quit."))

        #clean it
        query = query.translate(str.maketrans('\n',' ', string.punctuation)).lower()

        #if it's in the dictionary, print the values
        if query in dict.keys():
            print("The term '"+str(query)+"' is found in the following pages: \n" + str(dict[str(query)]))

        #if they quit, break the loop
        if query == "q":
            break

        #if it's not in the dictionary, print that there are no results. 
        if query not in dict.keys():
            print("No Results Found")

In [27]:
#Test the Search Query Interface:
SearchQueryInterface()

The term 'the' is found in the following pages: 
['index.html', 'information.html', 'ISchool.html', 'MIMS.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html', 'SouthHall.html', 'Campanile.html']
No Results Found
The term 'any' is found in the following pages: 
['index.html', 'UCBerkeley.html']
The term 'berkeley' is found in the following pages: 
['information.html', 'Berkeley.html', 'ISchool.html', 'CityOfBerkeley.html', 'UCBerkeley.html', 'BerkeleyCollege.html']
No Results Found
The term 'many' is found in the following pages: 
['information.html']
The term 'many' is found in the following pages: 
['information.html']
